<a href="https://colab.research.google.com/github/SanghunOh/study_machinglearning/blob/main/codes/yanginsuk/%EA%B3%BC%EC%A0%9C_2_2_iris_KNN%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
cd /content/drive/MyDrive/yanginsuk

/content/drive/MyDrive/yanginsuk


In [4]:
import scipy.io
mat = scipy.io.loadmat('./matlab_iris_shuffle.mat')
type(mat), mat.keys()

(dict,
 dict_keys(['__header__', '__version__', '__globals__', 'iris_class', 'iris_data']))

In [5]:
import numpy as np

In [6]:
for key in list(mat.keys()):
  result = None
  if isinstance(mat[key], np.ndarray):
    result = mat[key].shape
  else :
    result = mat[key]
  print('key name:{}, result:{}, data type:{}'.format(key ,result, type(mat[key])))

key name:__header__, result:b'MATLAB 5.0 MAT-file, written by Octave 5.2.0, 2020-03-26 12:54:05 UTC', data type:<class 'bytes'>
key name:__version__, result:1.0, data type:<class 'str'>
key name:__globals__, result:[], data type:<class 'list'>
key name:iris_class, result:(150, 1), data type:<class 'numpy.ndarray'>
key name:iris_data, result:(150, 4), data type:<class 'numpy.ndarray'>


In [7]:
class_type, class_count = np.unique(mat['iris_class'],return_counts=True)
class_type, class_count

(array([array([[1.]]), array([[2.]]), array([[3.]])], dtype=object),
 array([50, 50, 50]))

In [8]:
import pandas as pd

In [9]:
iris_pd = pd.DataFrame(mat['iris_data'])
iris_pd.info()
# iris_pd.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       150 non-null    object
 1   1       150 non-null    object
 2   2       150 non-null    object
 3   3       150 non-null    object
dtypes: object(4)
memory usage: 4.8+ KB


In [10]:
# dtype 일치 위해 변환
iris_data = mat['iris_data'].astype('float64')
iris_class = mat['iris_class'].astype('int32')

In [11]:
# iris = np.hstack([mat['iris_data'], mat['iris_class']])
iris = np.concatenate([iris_data, iris_class],axis=1)
iris.shape
iris.astype
iris = iris[iris[:, 4].argsort()] 

In [12]:
iris[3,0:5], np.linalg.norm(iris[2:4,0:4] - iris[3:5,0:4], axis=1)

(array([5. , 3.2, 1.2, 0.2, 1. ]), array([0.64031242, 0.74161985]))

In [13]:
print("data의 처음 다섯 행:\n{}".format(iris_data[:5]))
print("data의 처음 다섯 행:\n{}".format(iris_class[:5]))
print("iris 처음 다섯 행:\n{}".format(iris[:,0:5]))

data의 처음 다섯 행:
[[6.6 3.  4.4 1.4]
 [5.2 3.5 1.5 0.2]
 [6.8 3.  5.5 2.1]
 [5.9 3.2 4.8 1.8]
 [7.7 3.8 6.7 2.2]]
data의 처음 다섯 행:
[[2]
 [1]
 [3]
 [2]
 [3]]
iris 처음 다섯 행:
[[5.7 4.4 1.5 0.4 1. ]
 [4.8 3.4 1.9 0.2 1. ]
 [5.  3.5 1.6 0.6 1. ]
 [5.  3.2 1.2 0.2 1. ]
 [4.3 3.  1.1 0.1 1. ]
 [5.5 3.5 1.3 0.2 1. ]
 [5.2 4.1 1.5 0.1 1. ]
 [5.  3.3 1.4 0.2 1. ]
 [4.7 3.2 1.6 0.2 1. ]
 [4.6 3.4 1.4 0.3 1. ]
 [5.  3.5 1.3 0.3 1. ]
 [4.4 3.2 1.3 0.2 1. ]
 [4.8 3.  1.4 0.1 1. ]
 [5.4 3.9 1.7 0.4 1. ]
 [5.1 3.8 1.6 0.2 1. ]
 [5.4 3.4 1.7 0.2 1. ]
 [5.  3.6 1.4 0.2 1. ]
 [5.7 3.8 1.7 0.3 1. ]
 [5.1 3.4 1.5 0.2 1. ]
 [4.4 3.  1.3 0.2 1. ]
 [4.4 2.9 1.4 0.2 1. ]
 [5.8 4.  1.2 0.2 1. ]
 [4.8 3.1 1.6 0.2 1. ]
 [5.3 3.7 1.5 0.2 1. ]
 [5.  3.  1.6 0.2 1. ]
 [5.4 3.7 1.5 0.2 1. ]
 [5.1 3.8 1.5 0.3 1. ]
 [5.1 3.5 1.4 0.3 1. ]
 [4.8 3.  1.4 0.3 1. ]
 [5.2 3.4 1.4 0.2 1. ]
 [4.7 3.2 1.3 0.2 1. ]
 [4.6 3.2 1.4 0.2 1. ]
 [5.1 3.7 1.5 0.4 1. ]
 [5.4 3.4 1.5 0.4 1. ]
 [5.2 3.5 1.5 0.2 1. ]
 [4.9 3.  1.4 0.2 1. ]
 [5.1 

In [44]:
from pandas._libs.algos import ensure_uint8
import math
kernel_count = 1
error_count = 0
K = 30
lcnt=150
for i in range(len(iris)):
  distance_norm = np.linalg.norm(iris[:,0:4] - iris[i,0:4] , axis=1)
  #print("item 처음 다섯 행:\n{}".format(distance_norm))
  #kernel_stack = np.stack((distance_norm, iris[:,4]), axis=1)
  kernel_stack = np.stack((distance_norm, np.array(range(150))), axis=1)
  kernel_items = kernel_stack[kernel_stack[:,0].argsort()]  # 거리 계산 column 기준 가까운 거리로 정렬
  #if (i==50 or i==51 or i==49):  print("kernel_items :\n"+format(kernel_items)+":i:%d",i)
  class_types = np.full((3),0)
  #print("class_types 처음 000 행:\n"+format(class_types)+":i:%d",i)
  for n in range(K):
    #if (i==50):  print("kernel_items-1 행:\n{}".format(kernel_items[n,1]))
    if(int(kernel_items[n,1])<=49): class_types[0] = class_types[0] + 1
    if(int(kernel_items[n,1])>99): class_types[2] = class_types[2] + 1
    if(int(kernel_items[n,1])>49 and int(kernel_items[n,1])<=99): class_types[1] = class_types[1] + 1
  #if (i==50):  print("class_types-2 행:\n{}".format(class_types))
  max_val = np.argmax(class_types)
  #if (i==50):  print("max_val-3 행:{}",max_val)
  #if (i==50):  print("floor 처음 222 행:{}",(math.floor((i-1)/50)+1))
  if((max_val) != (math.floor((i)/50))):
    #print("에러-max_val:",(max_val+1))
    #print("에러-i:",i)
    #print("에러-floor:",math.floor((i)/50))
    error_count = error_count + 1
    #print("에러-error_count:",error_count)
  pass
#kernel_stack[0:10,:]
print('error rate : {}'.format(error_count/lcnt))
print('error rate : {}'.format(error_count))



error rate : 0.04666666666666667
error rate : 7


In [47]:
def knn_classtify(kernel_count):
  _kernel_count = kernel_count
  error_count = 0
  K = 30
  lcnt=150
  for i in range(len(iris)):
    distance_norm = np.linalg.norm(iris[:,0:4] - iris[i,0:4] , axis=1)
    #print("item 처음 다섯 행:\n{}".format(distance_norm))
    #kernel_stack = np.stack((distance_norm, iris[:,4]), axis=1)
    kernel_stack = np.stack((distance_norm, np.array(range(150))), axis=1)
    kernel_items = kernel_stack[kernel_stack[:,0].argsort()]  # 거리 계산 column 기준 가까운 거리로 정렬
    #if (i==50 or i==51 or i==49):  print("kernel_items :\n"+format(kernel_items)+":i:%d",i)
    class_types = np.full((3),0)
    #print("class_types 처음 000 행:\n"+format(class_types)+":i:%d",i)
    for n in range(_kernel_count):
      #if (i==50):  print("kernel_items-1 행:\n{}".format(kernel_items[n,1]))
      if(int(kernel_items[n,1])<=49): class_types[0] = class_types[0] + 1
      if(int(kernel_items[n,1])>99): class_types[2] = class_types[2] + 1
      if(int(kernel_items[n,1])>49 and int(kernel_items[n,1])<=99): class_types[1] = class_types[1] + 1
    #if (i==50):  print("class_types-2 행:\n{}".format(class_types))
    max_val = np.argmax(class_types)
    #if (i==50):  print("max_val-3 행:{}",max_val)
    #if (i==50):  print("floor 처음 222 행:{}",(math.floor((i-1)/50)+1))
    if((max_val) != (math.floor((i)/50))):
      #print("에러-max_val:",(max_val+1))
      #print("에러-i:",i)
      #print("에러-floor:",math.floor((i)/50))
      error_count = error_count + 1
  return error_count/lcnt

In [48]:
print(knn_classtify(1))
print(knn_classtify(5))
print(knn_classtify(10))
print(knn_classtify(15))
print(knn_classtify(20))
print(knn_classtify(25))
print(knn_classtify(30))

0.0
0.03333333333333333
0.02
0.013333333333333334
0.02
0.02
0.04666666666666667
